In [ ]:
import pandas as pd
import numpy as np

data1 = pd.read_csv('labeledTrainData.tsv', sep='\t', encoding = 'ISO-8859-1')
data1['review'] = data1['review'].str.replace('<br /><br />', ' ')

data2 = pd.read_csv('unlabeledTrainData.tsv', sep='\t', encoding = 'ISO-8859-1', error_bad_lines=False)
data2['review'] = data2['review'].str.replace('<br /><br />', ' ')

data3 = pd.read_csv('testData.tsv', sep='\t', encoding = 'ISO-8859-1')
data3['review'] = data3['review'].str.replace('<br /><br />', ' ')
sentences = list(data1['review']) + list(data3['review']) + list(data2['review'])

In [75]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim
documents = [TaggedDocument(gensim.utils.simple_preprocess(doc), [i]) for i, doc in enumerate(sentences)]

In [76]:
from random import shuffle
doc_list = documents[:]
shuffle(doc_list)

In [78]:
model = Doc2Vec(dm=0, vector_size=100, negative=5, hs=0, min_count=2, sample=0, epochs=20, workers=8)
model.build_vocab(documents)
model.train(doc_list, total_examples=len(doc_list), epochs=model.epochs)
model.save('./document.bin')

In [80]:
inferred_docvec = model.infer_vector(documents[1].words)
print('%s:\n %s' % (model, model.docvecs.most_similar([inferred_docvec], topn=3)))

Doc2Vec(dbow,d100,n5,mc2,t8):
 [(1, 0.9792658686637878), (59037, 0.5311046242713928), (14676, 0.5283793807029724)]


In [ ]:
model.infer_vector('i love you')

In [81]:
X_train = []
y_train = list(data1['sentiment'])
for i in range(len(y_train)):
    X_train.append(model[i])
print(len(X_train))

20000


In [87]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
clf = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(penalty="l1", dual=False, tol=1e-3))),
  ('classification', LinearSVC(penalty="l2"))])
clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('feature_selection', SelectFromModel(estimator=LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l1', random_state=None, tol=0.001,
     verbose=0),
        max_features=None, no...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [82]:
from sklearn.linear_model import LogisticRegression 
clf = LogisticRegression(solver='lbfgs')
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [103]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(10,2), random_state=1)
clf.fit(X_train[:18000], y_train[:18000])

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [104]:
clf.score(X_train[18000:], y_train[18000:])

0.8725

In [83]:
test_data = data3['review']
X_test = []
for i in range(len(test_data)):
    X_test.append(model[20000+i])
print(len(X_test))

5000


In [97]:
result = clf.predict(X_test)

In [98]:
test_id = list(data3['id'])
out = open('submission.csv', 'w')
out.write('id,sentiment\n')
for i in range(len(result)):
    out.write(test_id[i] + ',' + str(result[i]) + '\n')